# Porous Media Flow Simulator


This is a porous media flow simulator that I made. It is used to simulate flow in a reservoir. 
The simulator can handle single phase and two phase flow, as well as vertical wells, boundary conditions and different solvers. Here I will give an overview of the simulator. Examples of the simulator in action can be found [here](http://nbviewer.jupyter.org/github/JaimeLeal/projects/blob/master/Simulator_One_Phase_Flow.ipynb) and [here](http://nbviewer.jupyter.org/github/JaimeLeal/projects/blob/master/Simulator_Two_Phase_Flow.ipynb).

This reservoir simulator was developed as a final project for the class "Advanced Numerical Methods for Reservoir Simulation". 
Before working on this simulator I had some experience with [Matlab Reserovir Simulation Toolbox (MRST)](https://www.sintef.no/projectweb/mrst/ "MRST") , and some aspects of my simulator were inspired by it. The idea for the `UnitRegistry` class,  I got it from  [pint](https://pint.readthedocs.io/en/0.6/tutorial.html).

The simulator consists of four modules. Two simulator modules, `simulator.py` and `two_phase_flow.py`, one plotting module `simplots.py`, and one units and constants module, `units.py`.  


## Overview

- Units and constants
- Discretization : uniform Cartesian grid.
- Fluids: single phase and two phase fluids.
- Wells : vertical injectors and producers.
- Boundary conditions : Dirichlet constant pressure and no-flow boundary conditions.
- Solving Methods : Lagging Coefficients Method (LCM) and Fully Implicit Method (FIM).
- Linear solvers : direct and iterative solvers.

In [8]:
from modules.simulator.simulator import *
from modules.simulator.two_phase_flow import *
from modules.simulator.simplots import *
from modules.simulator.units import UnitRegistry
import numpy as np

## Units module

There are two classes in the `units` module, the `UnitRegistry` and `Constants`.
The `UnitRegistry` allows us to input the parameters in the International System of Units (SI) or in Field Units. All inputs are converted automatically to SI. The output of the simulator is also in SI units, and then it can be converted to the desired units with the help of this module.

The `Constants` class is used internally by the simulator, and contains physical constants such as gravity.

In [9]:
# New units registry
u = UnitRegistry()

distance = 3 * u.meter
print ("Distance is {:.3f} meters".format(distance))

# The units are converted to SI
distance = 3 * u.feet
print ("{:.3f} feet are {:.3f} meters".format(3, distance))

# There are prefixes
print("Centi: {:f}".format(u.centi))

# The units can be mixed together. 
perm = 1 * u.milli * u.darcy
print("1 millidarcy is {:.3E} Pa-s".format(perm))

Distance is 3.000 meters
3.000 feet are 0.914 meters
Centi: 0.010000
1 millidarcy is 9.869E-16 Pa-s


## Grid

The discretization of the partial differential equations was done using finite differences over a uniform cartesian grid. To specify the grid we use an instance of the class `uniformCartesianGrid`. We can create a grid with any number of layers.

In [10]:
# Create grid with 6 layers

# Number of cells in each direction
Nx, Ny, Nz = np.array([15, 15, 6])
# Dimension of each cell
Sx, Sy, Sz = np.array([30, 30, 30]) * u.feet
# Depth of each cell
depth = np.hstack((Sz * 5 * np.ones([Nx * Ny, ]),
                   Sz * 4 * np.ones([Nx * Ny, ]),
                   Sz * 3 * np.ones([Nx * Ny, ]),
                   Sz * 2 * np.ones([Nx * Ny, ]),
                   Sz * 1 * np.ones([Nx * Ny, ]),
                   Sz * 0 * np.ones([Nx * Ny, ])))

grid = uniformCartesianGrid(Nx, Ny, Nz, Sx, Sy, Sz, depth)

print("Number of cells {}.".format(grid.cellnumber))
print("Cell size {} meters.".format(grid.cellsize))
print("Cell volume {:.3f} m^3".format(grid.cellvolume))
print("Grid dimensions {}".format(grid.dim))

Number of cells 1350.
Cell size [ 9.144  9.144  9.144] meters.
Cell volume 764.555 m^3
Grid dimensions [15 15  6]


## Rock 
The characteristics of the rock are set in a `Rock` object. The rock has a constant compressibility for all the cells. The porosity and permeability fields are numpy arrays with the property value for each cell. And there is a parameter for the porosity as a function of compressibility. 

In [11]:
# Create rock
# Random porosiy and permeability fields.
poro = np.random.random_sample(grid.cellnumber)
perm = 10 * np.random.random_sample(grid.cellnumber) * u.milli * u.darcy

# Rock compressibility is equal  and constant for all cells.
cr = 3E-6 / u.psi
# The porosity is a function of rock compressibility
porofunc = lambda p: poro * np.exp(cr * (p - 2800 * u.psi))
rock = Rock(perm, poro, cr, porofunc)

## Fluid
There are two classes to specify a fluid in the simulator, `singleFluid` and `blackOil`.

### Single phase flow

`singleFluid` is for a single phase fluid with constant compressibility. The other properties, viscosity, density and formation volume factor, are functions and are provided by the user. The only requirement is that they are a function of pressure in Pascals.

In [5]:
#Example fluid
cf = 5E-5 / u.psi
# The functions should be in the correct units
miu = lambda p:  1.0 * np.exp(5E-5 * (p/u.psi - 2800)) * u.centi * u.poise
rho = lambda p: 800 * np.exp(cf * (p - 15 * u.psi) )  * u.kilogram/ u.meter**3
fvf = lambda p: 1.0 * np.exp(- cf * (p - 15 * u.psi) )   # adimensional
fluid = singleFluid(miu, rho, fvf, cf)

### Two phase flow
The class `blackOil` implements a two phase fluid, with the first phase being oil and the second one being water .The class is called `blackOil` even tough it is lacking support for gas flow. The parameters of the object are tuples of the viscosity, permeability, formation volume factor, and compressibility of the liquid. As wells as a function for the capillary pressure.

`fluid = blackOil((miu_o, miu_w), (rho_o, rho_w), (fvf_o, fvf_w), (cf_o, cf_w),(krofunc, krwfunc), pc)`


Just as with the `singleFluid`, the compressibility is a constant and viscosity, density and formation volume factor are a function of pressure. Relative permeability and capillary pressure are functions of water saturation.

## Plotting  module

The `simplots` module has several functions to plot the data that is generated with the simulator. These functions save the images to a folder specified by the user. For example,  `plotCellValueS2D`, plots each individual layer as a subplot with a single color bar.

In [6]:
# Plot rock properties
plotCellValues2D(grid, rock.poro, 'viridis', 
                 np.min(rock.poro), np.max(rock.poro), 
                 title='Porosity', filename = ".\images\poro")

permtoplot = np.log(rock.perm/u.darcy/u.milli)
plotCellValues2D(grid, permtoplot, 'magma', np.min(permtoplot), np.max(permtoplot), 
  title='Permeability [log(mD)]', filename = ".\images\perm")

C:\Users\Jaime\Anaconda3\lib\site-packages\matplotlib\figure.py:1718: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


<img src=".\images\poro.png">
<img src=".\images\perm.png">

## Wells: vertical injectors and producers

We can also add vertical wells to the reservoir. The producer wells are specified by setting the bottom hole pressure (bhp) and the injector wells are handled as point source terms  by setting the injection rate. 

The producer wells follow the Peaceman model and are assumed to be in the center of the cell and completed in a single layer.
For this example, we will create a `Wells` object with one  injector and one producer.

In [18]:
# First we initialize the Wells
W = Wells(grid, rock, fluid)

# Producer
rw = 0.3 * u.feet
location = 0
bhp = 2500 * u.psi
skin = 0

W.add_vertical_well(rw, location, bhp, skin)

# Injector


## Boundary Conditions
By default, all the boundaries of the reservoir are no-flow boundaries. With the `function` we can also set a constant pressure condition in a boundary. Currently, the function only works with the Lagging Coefficient Method. With any other method the boundaries can only be no flow.

## Well schedule
With the well schedule we can set the rate and the pressure of each well.

## Solvers

### Lagging Coefficients

### Fully Implicit Method

## Examples

- [Single Phase Flow Example](http://nbviewer.jupyter.org/github/JaimeLeal/projects/blob/master/Simulator_One_Phase_Flow.ipynb) 
    
- [Two Phase Flow Example](http://nbviewer.jupyter.org/github/JaimeLeal/projects/blob/master/Simulator_Two_Phase_Flow.ipynb).

In [10]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# I am using the style sheet found here  Lorena Barba https://github.com/barbagroup/CFDPython/blob/master/styles/custom.css